In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler





# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")
data.shape
data.tail(1)

data = data.iloc[:10747]

data.tail(1)


data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()



#data["date"] = data["date"].astype(float)
data["open"] = data["open"].astype(float)
data["high"] = data["high"].astype(float)
data["low"] = data["low"].astype(float)
data["volume"] = data["volume"].astype(float)
data["adjusted_close"] = data["adjusted_close"].astype(float)
data["change_percent"] = data["change_percent"].astype(float)
data["avg_vol_20d"] = data["avg_vol_20d"].astype(float)

data["close"] = data["close"].astype(float)

#print(data.tail(1))

In [10]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


# Convert data to numpy arrays
x_data = data[[ 'open', 'high', 'low', 'volume','adjusted_close', 'change_percent', 'avg_vol_20d']].values
y_data = data["close"].values.reshape(-1, 1)  # Reshape y_data if it's 1D



# Convert data to Tensors
x_feature_tensors = torch.tensor(x_data, dtype=torch.float32)
y_feature_tensors = torch.tensor(y_data, dtype=torch.float32)


# Split the data into training and testing sets
train_size = int(len(x_feature_tensors) * 0.67)
x_train, x_test = x_feature_tensors[:train_size], x_feature_tensors[train_size:]
y_train, y_test = y_feature_tensors[:train_size], y_feature_tensors[train_size:]



# Initialize the scaler
scaler = MinMaxScaler()

# fitting the MinMaxScaler to training data using fit_transform and then
#  test data only transform
x_train_scaled = scaler.fit_transform(x_train.detach().numpy())
x_test_scaled = scaler.transform(x_test.detach().numpy())  # Use transform, not fit_transform

y_train_scaled = scaler.fit_transform(y_train.detach().numpy())
y_test_scaled = scaler.transform(y_test.detach().numpy())  # Use transform, not fit_transform





In [11]:

learning_rate = 0.001
window_size = 50
test_window_size = 50
hidden_dim = 64
n_layers = 4
batch_evaluation_frequency = 10
epochs = len(x_train_scaled) - window_size
batch_size = 1
input_size = x_train_scaled.shape[1]  # Input size based on your dataset
output_size = 1  # Output size (for regression task)

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers, output_size):
        super(LSTMModel, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x, hidden):
        batch_size = x.size(0)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden


In [12]:
import torch
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import train_test_split
import random
import pandas as pd

# Assuming x_train_scaled, y_train_scaled, x_val_scaled, y_val_scaled, x_test_scaled, y_test_scaled,
# input_size, output_size, test_window_size, scaler are available

# Splitting the dataset into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train_scaled, y_train_scaled, test_size=0.2, shuffle=False)

# Hyperparameter search space
search_space = {
    'learning_rate': [0.00001, 0.00005, 0.0001],
    'window_size': [3, 5, 6],
    'hidden_dim': [64, 128, 256],
    'n_layers': [10, 11, 12],
    'batch_evaluation_frequency': [4, 5, 6]
}

# Best Parameters: {'learning_rate': 5e-05, 'window_size': 5, 'hidden_dim': 128,
# 'n_layers': 11, 'batch_evaluation_frequency': 7}

# Best Parameters: {'learning_rate': 0.0001, 'window_size': 10, 'hidden_dim': 256,
# 'n_layers': 10, 'batch_evaluation_frequency': 5}



# Number of random search iterations
num_iterations = 10

best_params = None

for _ in range(num_iterations):
    params = {
        'learning_rate': random.choice(search_space['learning_rate']),
        'window_size': random.choice(search_space['window_size']),
        'hidden_dim': random.choice(search_space['hidden_dim']),
        'n_layers': random.choice(search_space['n_layers']),
        'batch_evaluation_frequency': random.choice(search_space['batch_evaluation_frequency'])
    }

    model = LSTMModel(input_size, params['hidden_dim'], params['n_layers'], output_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

    # Training using Walk-Forward Validation
    for i in range(params['window_size'], len(x_train)):
        optimizer.zero_grad()

        start_idx = i - params['window_size']
        end_idx = i
        x_window = torch.tensor(x_train[start_idx:end_idx], dtype=torch.float32)
        y_window = torch.tensor(y_train[start_idx:end_idx], dtype=torch.float32)
        x_window = x_window.view(1, params['window_size'], input_size)

        hidden = model.init_hidden(1)
        outputs, hidden = model(x_window, hidden)
        loss = criterion(outputs, y_window)

        loss.backward()
        optimizer.step()

        if i % params['batch_evaluation_frequency'] == 0:
            with torch.no_grad():
                x_val_window = torch.tensor(x_val[:params['window_size']], dtype=torch.float32)
                y_val_window = torch.tensor(y_val[:params['window_size']], dtype=torch.float32)
                x_val_window = x_val_window.view(1, params['window_size'], input_size)

                hidden = model.init_hidden(1)
                val_outputs, _ = model(x_val_window, hidden)
                val_loss = criterion(val_outputs, y_val_window)

                if i % 1000 == 0:  # Print every 1000 iterations
                    print(f"Iteration {i}, Train Loss: {loss.item()}, Validation Loss: {val_loss.item()}")


    if best_params is None:
        best_params = params
        best_params['val_loss'] = val_loss.item()
    elif val_loss < best_params['val_loss']:
        best_params = params
        best_params['val_loss'] = val_loss.item()


"""
num_future_days = 5

# Initialize x_test_window with the last values from the validation set
x_test_window = torch.tensor(x_val[-params['window_size']:], dtype=torch.float32).view(1, params['window_size'], input_size)

for day in range(num_future_days):
    with torch.no_grad():
        # Use the last predicted values as input for the next prediction
        last_test_window_tensor = x_test_window
        hidden = model.init_hidden(1)
        predicted_close_scaled, _ = model(last_test_window_tensor, hidden)

        # Extract the scalar value from the predicted_close_scaled tensor
        predicted_close_scalar = predicted_close_scaled.item()

        # Update the test window with the scalar value
        #x_test_window = torch.cat((x_test_window[:, 1:, :], torch.tensor([[[predicted_close_scalar]]], dtype=torch.float32)), dim=1)
        x_test_window = torch.cat((x_test_window[:, :, 1:], torch.tensor([[[predicted_close_scalar]]], dtype=torch.float32)), dim=2)


        # Convert the predicted value to the original scale if needed
        predicted_close = scaler.inverse_transform([[predicted_close_scalar]])

        # Print or store the predicted value for the current day
        print(f"Predicted Close for Day {day + 1}: {predicted_close[0][0]}")

print(f"Best Parameters: {best_params}")


"""


c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([5, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Iteration 1000, Train Loss: 9.03335603652522e-05, Validation Loss: 3.187233960488811e-05
Iteration 2000, Train Loss: 0.00016266644524876028, Validation Loss: 0.007436024956405163
Iteration 3000, Train Loss: 0.00029523862758651376, Validation Loss: 0.013675990514457226
Iteration 4000, Train Loss: 1.761840576364193e-05, Validation Loss: 1.0649879186530598e-05
Iteration 5000, Train Loss: 0.000991955865174532, Validation Loss: 0.02531951293349266
Iteration 1000, Train Loss: 0.0002084744191961363, Validation Loss: 0.00011452690523583442
Iteration 2000, Train Loss: 0.0002718893811106682, Validation Loss: 0.008175069466233253
Iteration 3000, Train Loss: 0.00029494293266907334, Validation Loss: 0.013710128143429756
Iteration 4000, Train Loss: 3.3245247323065996e-05, Validation Loss: 3.3632288250373676e-05
Iteration 5000, Train Loss: 0.0010800770251080394, Validation Loss: 0.024367591366171837
Iteration 1000, Train Loss: 7.177585212048143e-05, Validation Loss: 2.305101224919781e-05
Iteration 20

c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([6, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Iteration 1000, Train Loss: 0.00040262166294269264, Validation Loss: 0.000269543583272025
Iteration 2000, Train Loss: 0.00048125427565537393, Validation Loss: 0.009138894267380238
Iteration 3000, Train Loss: 0.00011017655924661085, Validation Loss: 0.014905576594173908
Iteration 4000, Train Loss: 5.37687010364607e-05, Validation Loss: 1.5725339835626073e-05
Iteration 5000, Train Loss: 0.00029185667517594993, Validation Loss: 0.0320160873234272
Iteration 1000, Train Loss: 7.166488649090752e-05, Validation Loss: 2.4153188860509545e-05
Iteration 2000, Train Loss: 0.00017196421686094254, Validation Loss: 0.007424874696880579
Iteration 3000, Train Loss: 0.00022497221652884036, Validation Loss: 0.013632893562316895
Iteration 4000, Train Loss: 3.0709223210578784e-05, Validation Loss: 5.0775775889633223e-05
Iteration 5000, Train Loss: 0.001139795291237533, Validation Loss: 0.02313563972711563


c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([3, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Iteration 1000, Train Loss: 0.00011962981807300821, Validation Loss: 0.00010496858158148825
Iteration 2000, Train Loss: 0.0006779047544114292, Validation Loss: 0.010713930241763592
Iteration 3000, Train Loss: 2.8394124456099235e-05, Validation Loss: 0.019662434235215187
Iteration 4000, Train Loss: 0.0008034008205868304, Validation Loss: 0.0006578708998858929
Iteration 5000, Train Loss: 0.011008888483047485, Validation Loss: 0.07786264270544052
Iteration 1000, Train Loss: 0.00019668356981128454, Validation Loss: 0.00010512720473343506
Iteration 2000, Train Loss: 0.0003265512641519308, Validation Loss: 0.008380747400224209
Iteration 3000, Train Loss: 0.0002231731341453269, Validation Loss: 0.013674949295818806
Iteration 4000, Train Loss: 2.3877815692685544e-05, Validation Loss: 3.743471097550355e-05
Iteration 5000, Train Loss: 0.0007023843936622143, Validation Loss: 0.02552490495145321
Iteration 1000, Train Loss: 8.398994395975024e-05, Validation Loss: 2.8963268050574698e-05
Iteration 20

RuntimeError: Sizes of tensors must match except in dimension 2. Expected size 3 but got size 1 for tensor number 1 in the list.

In [32]:
import torch

# Assuming you have your x_test_window and predicted_close_scalar tensors
x_test_window = torch.randn(1, 3, 7)  # Example size, replace with your actual data
predicted_close_scalar = torch.randn(1, 1, 1)  # Example size, replace with your actual data

# Ensure the dimensions match for concatenation
predicted_close_scalar = predicted_close_scalar.expand(-1, 3, -1)

# Concatenate along the second dimension
new_x_test_window = torch.cat((x_test_window, predicted_close_scalar), dim=2)

# Print the sizes to verify
print("Size of x_test_window:", x_test_window.size())
print("Size of predicted_close_scalar:", predicted_close_scalar.size())
print("Size of new_x_test_window:", new_x_test_window.size())


Size of x_test_window: torch.Size([1, 3, 7])
Size of predicted_close_scalar: torch.Size([1, 3, 1])
Size of new_x_test_window: torch.Size([1, 3, 8])


In [67]:
import torch

# Assuming you have defined your model, scaler, and other necessary parameters

# Initialize x_test_window with the last values from the validation set
x_test_window = torch.tensor(x_val[-params['window_size']:], dtype=torch.float32).view(1, params['window_size'], input_size)

for day in range(num_future_days):
    with torch.no_grad():
        # Use the last predicted values as input for the next prediction
        last_test_window_tensor = x_test_window
        hidden = model.init_hidden(1)
        predicted_close_scalar, _ = model(last_test_window_tensor, hidden)

        # Ensure the correct shape for predicted_close_scalar
        predicted_close_scalar = predicted_close_scalar.view(1, 1, 1)

        # Create a new tensor with the desired shape [1, 3, 1]
        predicted_close_tensor = torch.zeros(1, 3, 1)
        predicted_close_tensor[:, :, 0] = predicted_close_scalar[:, 0, 0]

        print(f"Size of x_test_window: {x_test_window.size()}")
        print(f"Size of predicted_close_scalar: {predicted_close_scalar.size()}")

        # Update the test window by including the latest predicted value
        updated_window = torch.cat((x_test_window[:, :, 1:], predicted_close_tensor), dim=2)

        # Convert the predicted value to the original scale if needed
        predicted_close = scaler.inverse_transform(predicted_close_tensor[0].detach().numpy())

        # Print or store the predicted value for the current day
        print(f"Predicted Close for Day {day + 1}: {predicted_close[0]}")

        # Update x_test_window for the next iteration
        x_test_window = updated_window


Size of x_test_window: torch.Size([1, 3, 7])
Size of predicted_close_scalar: torch.Size([1, 1, 1])
Predicted Close for Day 1: [21.143133]
Size of x_test_window: torch.Size([1, 3, 7])
Size of predicted_close_scalar: torch.Size([1, 1, 1])
Predicted Close for Day 2: [21.14314]
Size of x_test_window: torch.Size([1, 3, 7])
Size of predicted_close_scalar: torch.Size([1, 1, 1])
Predicted Close for Day 3: [21.143135]
Size of x_test_window: torch.Size([1, 3, 7])
Size of predicted_close_scalar: torch.Size([1, 1, 1])
Predicted Close for Day 4: [21.143139]
Size of x_test_window: torch.Size([1, 3, 7])
Size of predicted_close_scalar: torch.Size([1, 1, 1])
Predicted Close for Day 5: [21.14314]


In [63]:
num_future_days = 5

# Initialize x_test_window with the last values from the validation set
x_test_window = torch.tensor(x_val[-params['window_size']:], dtype=torch.float32).view(1, params['window_size'], input_size)

for day in range(num_future_days):
    with torch.no_grad():
        # Use the last predicted values as input for the next prediction
        last_test_window_tensor = x_test_window
        hidden = model.init_hidden(1)
        predicted_close_scalar, _

        # Ensure the correct shape for predicted_close_scalar
        predicted_close_scalar = predicted_close_scalar.view(1, -1, 1)

        # Explicitly reshape to match the desired size [1, 3, 1]
        predicted_close_scalar = predicted_close_scalar.expand(1, 3, 1)

        print(f"Size of x_test_window: {x_test_window.size()}")
        print(f"Size of predicted_close_scalar: {predicted_close_scalar.size()}")

        # Update the test window by including the latest predicted value
        updated_window = torch.cat((x_test_window[:, :, 1:], predicted_close_scalar), dim=2)

        # Convert the predicted value to the original scale if needed
        predicted_close = scaler.inverse_transform([[predicted_close_scalar[:,:,0].item()]])

        # Print or store the predicted value for the current day
        print(f"Predicted Close for Day {day + 1}: {predicted_close[0]}")

        # Update x_test_window for the next iteration
        x_test_window = updated_window


Size of x_test_window: torch.Size([1, 3, 7])
Size of predicted_close_scalar: torch.Size([1, 3, 1])


RuntimeError: a Tensor with 3 elements cannot be converted to Scalar

In [ ]:
num_future_days = 5

# Initialize x_test_window with the last values from the validation set
x_test_window = torch.tensor(x_val[-params['window_size']:], dtype=torch.float32).view(1, params['window_size'], input_size)

for day in range(num_future_days):
    with torch.no_grad():
        # Use the last predicted values as input for the next prediction
        last_test_window_tensor = x_test_window
        hidden = model.init_hidden(1)
        predicted_close_scaled, _ = model(last_test_window_tensor, hidden)


        print(f"Size of x_test_window: {x_test_window.size()}")
        print(f"Size of predicted_close_scaled: {predicted_close_scaled.size()}")


        # Update the test window by including the latest predicted value
        updated_window = torch.cat((x_test_window[:, :, 1:], predicted_close_scaled), dim=2)

        # Convert the predicted value to the original scale if needed
        predicted_close = scaler.inverse_transform([[predicted_close_scaled[:,:,0].item(), predicted_close_scaled[:,:,1].item(), predicted_close_scaled[:,:,2].item()]])

        # Print or store the predicted value for the current day
        print(f"Predicted Close for Day {day + 1}: {predicted_close[0]}")

print(f"Best Parameters: {best_params}")




Size of x_test_window: torch.Size([1, 3, 7])
Size of predicted_close_scaled: torch.Size([1, 1])


RuntimeError: Tensors must have same number of dimensions: got 3 and 2

In [53]:
print("Size of x_test_window:", x_test_window.size())
print("Size of predicted_close_scalar:", predicted_close_scalar.size())



Size of x_test_window: torch.Size([1, 3, 7])
Size of predicted_close_scalar: torch.Size([1, 7, 1])


In [41]:
num_future_days = 5

# Initialize x_test_window with the last values from the validation set
x_test_window = torch.tensor(x_val[-params['window_size']:], dtype=torch.float32).view(1, params['window_size'], input_size)

for day in range(num_future_days):
    with torch.no_grad():
        # Use the last predicted values as input for the next prediction
        last_test_window_tensor = x_test_window
        hidden = model.init_hidden(1)
        predicted_close_scaled, _ = model(last_test_window_tensor, hidden)

        # Extract the scalar value from the predicted_close_scaled tensor
        predicted_close_scalar = predicted_close_scaled.item()





        # Reshape predicted_close_scalar to match the dimensions of x_test_window

        # Update the test window by including the latest predicted value
        updated_window = torch.cat((x_test_window[:, :, 1:], predicted_close_scalar), dim=2)





        # Convert the predicted value to the original scale if needed
        predicted_close = scaler.inverse_transform([[predicted_close_scalar]])

        # Print or store the predicted value for the current day
        print(f"Predicted Close for Day {day + 1}: {predicted_close[0][0]}")

print(f"Best Parameters: {best_params}")

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [35]:
# Initialize x_test_window with the last values from the validation set
x_test_window = torch.tensor(x_val[-params['window_size']:], dtype=torch.float32).view(1, params['window_size'], input_size)

for day in range(num_future_days):
    with torch.no_grad():
        # Use the last predicted values as input for the next prediction
        hidden = model.init_hidden(1)
        predicted_close_scaled, _ = model(x_test_window, hidden)

        # Extract the scalar value from the predicted_close_scaled tensor
        predicted_close_scalar = predicted_close_scaled.item()



       # Reshape predicted_close_scalar to match the dimensions of x_test_window
        predicted_close_scalar = torch.unsqueeze(torch.unsqueeze(torch.tensor(predicted_close_scalar, dtype=torch.float32), dim=0), dim=2)

        # Update the test window by including the latest predicted value
        updated_window = torch.cat((x_test_window, predicted_close_scalar), dim=2)




        # Convert the predicted value to the original scale if needed
        predicted_close = scaler.inverse_transform([[predicted_close_scalar]])

        # Print or store the predicted value for the current day
        print(f"Predicted Close for Day {day + 1}: {predicted_close[0][0]}")

        # Update x_test_window with the new window
        x_test_window = updated_window

# If needed, you can further update the model parameters based on the new data
# For example, you might want to retrain the model with the updated data

print(f"Best Parameters: {best_params}")


IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [21]:
# Assuming x_test_window is a PyTorch tensor and predicted_close_scalar is a scalar value
print("Size of x_test_window[:, :, :]:", x_test_window[:, :, :].size())
print("Size of torch.tensor([[[predicted_close_scalar]]]):", torch.tensor([[[predicted_close_scalar]]]).size())



Size of x_test_window[:, :, :]: torch.Size([1, 3, 7])
Size of torch.tensor([[[predicted_close_scalar]]]): torch.Size([1, 1, 1])


In [26]:
import torch

# Assuming you have your x_test_window and predicted_close_scalar tensors
x_test_window = torch.randn(1, 3, 7)  # Example size, replace with your actual data
predicted_close_scalar = torch.randn(1, 1, 1)  # Example size, replace with your actual data

# Ensure the dimensions match for concatenation
predicted_close_scalar = predicted_close_scalar.expand(-1, 3, -1)

# Concatenate along the second dimension
new_x_test_window = torch.cat((x_test_window, predicted_close_scalar), dim=2)

# Print the sizes to verify
print("Size of x_test_window:", x_test_window.size())
print("Size of predicted_close_scalar:", predicted_close_scalar.size())
print("Size of new_x_test_window:", new_x_test_window.size())


Size of x_test_window: torch.Size([1, 3, 7])
Size of predicted_close_scalar: torch.Size([1, 3, 1])
Size of new_x_test_window: torch.Size([1, 3, 8])


In [19]:
# Initialize x_test_window with the last values from the validation set
x_test_window = torch.tensor(x_val[-params['window_size']:], dtype=torch.float32).view(1, params['window_size'], input_size)

for day in range(num_future_days):
    with torch.no_grad():
        # Use the last predicted values as input for the next prediction
        hidden = model.init_hidden(1)
        predicted_close_scaled, _ = model(x_test_window, hidden)

        # Extract the scalar value from the predicted_close_scaled tensor
        predicted_close_scalar = predicted_close_scaled.item()

        # Update the test window by including the latest predicted value
        updated_window = torch.cat((x_test_window[:, :, :], torch.tensor([[[predicted_close_scalar]]], dtype=torch.float32)), dim=2)

        # Convert the predicted value to the original scale if needed
        predicted_close = scaler.inverse_transform([[predicted_close_scalar]])

        # Print or store the predicted value for the current day
        print(f"Predicted Close for Day {day + 1}: {predicted_close[0][0]}")

        # Update x_test_window with the new window
        x_test_window = updated_window

# If needed, you can further update the model parameters based on the new data
# For example, you might want to retrain the model with the updated data

print(f"Best Parameters: {best_params}")


RuntimeError: Sizes of tensors must match except in dimension 2. Expected size 3 but got size 1 for tensor number 1 in the list.

In [18]:
# Initialize x_test_window with the last values from the validation set
x_test_window = torch.tensor(x_val[-params['window_size']:], dtype=torch.float32).view(1, params['window_size'], input_size)

for day in range(num_future_days):
    with torch.no_grad():
        # Use the last predicted values as input for the next prediction
        hidden = model.init_hidden(1)
        predicted_close_scaled, _ = model(x_test_window, hidden)

        # Extract the scalar value from the predicted_close_scaled tensor
        predicted_close_scalar = predicted_close_scaled.item()

        # Update the test window by including the latest predicted value
        # Assuming x_test_window has shape (1, window_size, num_features)
        updated_window = torch.cat((x_test_window[:, :, 1:], torch.tensor([[[predicted_close_scalar]]], dtype=torch.float32)), dim=2)

        # Convert the predicted value to the original scale if needed
        predicted_close = scaler.inverse_transform([[predicted_close_scalar]])

        # Print or store the predicted value for the current day
        print(f"Predicted Close for Day {day + 1}: {predicted_close[0][0]}")

        # Update x_test_window with the new window
        x_test_window = updated_window

# If needed, you can further update the model parameters based on the new data
# For example, you might want to retrain the model with the updated data

print(f"Best Parameters: {best_params}")


RuntimeError: Sizes of tensors must match except in dimension 2. Expected size 3 but got size 1 for tensor number 1 in the list.

In [16]:
# Initialize x_test_window with the last values from the validation set
x_test_window = torch.tensor(x_val[-params['window_size']:], dtype=torch.float32).view(1, params['window_size'], input_size)

for day in range(num_future_days):
    with torch.no_grad():
        # Use the last predicted values as input for the next prediction
        hidden = model.init_hidden(1)
        predicted_close_scaled, _ = model(x_test_window, hidden)

        # Extract the scalar value from the predicted_close_scaled tensor
        predicted_close_scalar = predicted_close_scaled.item()

        # Update the test window by including the latest predicted value
        # Assuming x_test_window has shape (1, window_size, num_features)
        updated_window = torch.cat((x_test_window[:, :, 1:], torch.tensor([[[predicted_close_scalar]]], dtype=torch.float32)), dim=2)

        # Convert the predicted value to the original scale if needed
        predicted_close = scaler.inverse_transform([[predicted_close_scalar]])

        # Print or store the predicted value for the current day
        print(f"Predicted Close for Day {day + 1}: {predicted_close[0][0]}")

        # Update x_test_window with the new window
        x_test_window = updated_window

# If needed, you can further update the model parameters based on the new data
# For example, you might want to retrain the model with the updated data

print(f"Best Parameters: {best_params}")


RuntimeError: Sizes of tensors must match except in dimension 2. Expected size 3 but got size 1 for tensor number 1 in the list.

In [14]:
num_future_days = 5

# Initialize x_test_window with the last values from the validation set
x_test_window = torch.tensor(x_val[-params['window_size']:], dtype=torch.float32).view(1, params['window_size'], input_size)

for day in range(num_future_days):
    with torch.no_grad():
        # Use the last predicted values as input for the next prediction
        hidden = model.init_hidden(1)
        predicted_close_scaled, _ = model(x_test_window, hidden)

        # Extract the scalar value from the predicted_close_scaled tensor
        predicted_close_scalar = predicted_close_scaled.item()

        # Update the test window by including the latest predicted value
        x_test_window = torch.cat((x_test_window[:, :, 1:], torch.tensor([[[predicted_close_scalar]]], dtype=torch.float32)), dim=2)

        # Convert the predicted value to the original scale if needed
        predicted_close = scaler.inverse_transform([[predicted_close_scalar]])

        # Print or store the predicted value for the current day
        print(f"Predicted Close for Day {day + 1}: {predicted_close[0][0]}")

# If needed, you can further update the model parameters based on the new data
# For example, you might want to retrain the model with the updated data

print(f"Best Parameters: {best_params}")


RuntimeError: Sizes of tensors must match except in dimension 2. Expected size 3 but got size 1 for tensor number 1 in the list.

In [13]:
# Print dimensions of x_test_window
print("Dimensions of x_test_window:", x_test_window.size())

# Print dimensions of predicted_close_scalar
print("Dimensions of predicted_close_scalar:", torch.tensor([[[predicted_close_scalar]]], dtype=torch.float32).size())


Dimensions of x_test_window: torch.Size([1, 3, 7])
Dimensions of predicted_close_scalar: torch.Size([1, 1, 1])


In [ ]:
# Update the test window with the scalar value
predicted_close_tensor = torch.tensor([[[predicted_close_scalar]]], dtype=torch.float32)

# Print sizes of tensors
print("x_test_window[:, 1:, :].size():", x_test_window[:, 1:, :].size())
print("predicted_close_tensor.size():", predicted_close_tensor.size())

# Concatenate tensors
x_test_window = torch.cat((x_test_window[:, 1:, :], predicted_close_tensor), dim=1)


NameError: name 'predicted_close_scalar' is not defined

In [ ]:
real_value_31_07_2024 = pd.read_csv("../data/data/aapl_raw_data.csv").loc[10747, 'close']
real_value_01_08_2024 = pd.read_csv("../data/data/aapl_raw_data.csv").loc[10748, 'close']
real_value_02_08_2024 = pd.read_csv("../data/data/aapl_raw_data.csv").loc[10749, 'close']
print(f"real_value_31_07_2024: {real_value_31_07_2024}")
print("")
print(f"real_value_01_08_2024: {real_value_01_08_2024}")
print(f"real_value_02_08_2024: {real_value_02_08_2024}")

real_value_31_07_2024: 195.61

real_value_01_08_2024: 192.58
real_value_02_08_2024: 191.17


In [ ]:
# Update the test window with the scalar value
predicted_close_tensor = torch.tensor([[[predicted_close_scalar]]], dtype=torch.float32)

# Print sizes of tensors
print("x_test_window[:, 1:, :].size():", x_test_window[:, 1:, :].size())
print("predicted_close_tensor.size():", predicted_close_tensor.size())

# Concatenate tensors
#x_test_window = torch.cat((x_test_window[:, 1:, :], predicted_close_tensor), dim=1)


NameError: name 'predicted_close_scalar' is not defined

In [ ]:
x_test_window = torch.tensor(x_val[-params['window_size']:], dtype=torch.float32).view(1, params['window_size'], input_size)


# Print the dimensions of x_test_window[:, 1:]
print("Dimensions of x_test_window[:, 1:]:", x_test_window[:, 1:].shape)

# Extract the scalar value from the predicted_close_scaled tensor
predicted_close_scalar = predicted_close_scaled.item()

# Create an array with the scalar value
predicted_close_scalar_array = np.array([[predicted_close_scalar]])

# Print the dimensions of predicted_close_scalar_array
print("Dimensions of predicted_close_scalar_array:", predicted_close_scalar_array.shape)

# Update the test window with the scalar value
x_test_window = np.concatenate((x_test_window[:, 1:], predicted_close_scalar_array), axis=1)


Dimensions of x_test_window[:, 1:]: torch.Size([1, 2, 7])
Dimensions of predicted_close_scalar_array: (1, 1)


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)